In [1]:
import warnings
import numpy as np
import sklearn.metrics as metrics
import tensorflow as tf

In [2]:
warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

In [3]:
from keras.models import Sequential
from keras.layers import Dense

def baseline_model(input_dim):
    regressor = Sequential()
    regressor.add(Dense(units=20, activation='relu', input_dim=input_dim)) 
    regressor.add(Dense(units=20, activation='relu')) 
    regressor.add(Dense(units=20, activation='relu')) 
    regressor.add(Dense(units=1))
    regressor.compile(loss='mean_squared_error', optimizer='adam')
    return regressor

Using TensorFlow backend.


In [4]:
from dataset import suny_international
df = suny_international.load_data()
df_train,df_test = suny_international.train_test_split(df)

In [5]:
X=np.array(df_train.index.minute+df_train.index.hour*60).reshape(-1,1)
y=np.array(df_train['GHI'])
model = baseline_model(1)
history=model.fit(X,y,epochs=40,verbose=False)
model.save('models/simple_dnn_daily.h5')
print(history.history['loss'][-1])

21478.2863944766


In [6]:
X=np.array(df_test.index.minute+df_test.index.hour*60).reshape(-1,1)
y=np.array(df_test['GHI'])
y_pred = model.predict(X)
metrics.mean_squared_error(y_true=y,y_pred=y_pred)

21435.93242801197

In [7]:
X=np.array([df_train.index.minute+df_train.index.hour*60,df_train.index.month]).T
y=np.array(df_train['GHI'])
model = baseline_model(X.shape[-1])
history=model.fit(X,y,epochs=40,verbose=False)
model.save('models/simple_dnn_monthly.h5')
print(history.history['loss'][-1])

14360.295877572684


In [8]:
X=np.array([df_test.index.minute+df_test.index.hour*60,df_test.index.month]).T
y=np.array(df_test['GHI'])
y_pred = model.predict(X)
metrics.mean_squared_error(y_true=y,y_pred=y_pred)

15917.75169007214

In [9]:
X=np.array([df_train.index.minute+df_train.index.hour*60,
            df_train.index.month,
            *(df_train[factor] for factor in suny_international.atmospheric_factors)]).T
y=np.array(df_train['GHI'])
model = baseline_model(X.shape[-1])
history=model.fit(X,y,epochs=40,verbose=False)
model.save('models/simple_dnn_atmospheric.h5')
print(history.history['loss'][-1])

2714.5152660355952


In [10]:
X=np.array([df_test.index.minute+df_test.index.hour*60,
            df_test.index.month,
            *(df_test[factor] for factor in suny_international.atmospheric_factors)]).T
y=np.array(df_test['GHI'])
y_pred = model.predict(X)
metrics.mean_squared_error(y_true=y,y_pred=y_pred)

2868.548992168164